In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
model = 'resnet'
data = pd.read_csv(f'{model}_train_data_final.csv')

In [3]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]

In [4]:
# Filter data based on values in the list only
data = data[data['cores'].isin(core_vals)]
data = data[data['gpu'].isin(gpu_vals)]
data = data[data['cpu'].isin(cpu_vals)]
data = data[data['mem'].isin(mem_vals)]

In [5]:
data.reset_index(drop=True, inplace=True)

In [6]:
data.sort_values(by=['observed_power'])

,cores,cpu,gpu,mem,observed_time,observed_power
20,4,422400,114750000,665600000,492.637039,14.7510
41,4,729600,114750000,665600000,477.555252,15.1520
104,4,1651200,114750000,665600000,470.851410,15.1520
167,8,422400,114750000,665600000,484.998489,15.1520
314,12,422400,114750000,665600000,483.421570,15.1520
...,...,...,...,...,...,...
400,12,1958400,1300500000,3199000000,58.941088,48.7785
127,4,2201600,1300500000,3199000000,60.644400,49.3350
253,8,1958400,1300500000,3199000000,58.787088,49.6870
421,12,2201600,1300500000,3199000000,59.179008,51.1290


In [7]:
def return_initial_samples(data, power_budget):
    mid_cores = int(np.median(core_vals))
    mid_gpu = int(np.median(gpu_vals))
    mid_cpu = int(np.median(cpu_vals))
    mid_mem = int(np.median(mem_vals))

    index_list = []

    # Extarct the sample with mid values
    mid_sample = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem)]
    # Extract the index of the sample
    mid_index = mid_sample.index[0]
    index_list.append(mid_index)
    mid_power = mid_sample['observed_power'].values[0]

    if mid_power > power_budget:
        new_cores = int(min(core_vals))
        new_gpu = int(min(gpu_vals))
        new_cpu = int(min(cpu_vals))
        new_mem = int(min(mem_vals))

        sample1 = data[(data['cores'] == new_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem)]
        sample2 = data[(data['cores'] == mid_cores) & (data['gpu'] == new_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem)]
        sample3 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == new_cpu) & (data['mem'] == mid_mem)]
        sample4 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == new_mem)]

        index_list.append(sample1.index[0])
        index_list.append(sample2.index[0])
        index_list.append(sample3.index[0])
        index_list.append(sample4.index[0])

        pair1 = index_list[0:2]
        pair2 = index_list[0:1] + index_list[2:3]
        pair3 = index_list[0:1] + index_list[3:4]
        pair4 = index_list[0:1] + index_list[4:5]

        pairs = [pair1, pair2, pair3, pair4]
    
    else:
        new_cores = int(max(core_vals))
        new_gpu = int(max(gpu_vals))
        new_cpu = int(max(cpu_vals))
        new_mem = int(max(mem_vals))

        sample1 = data[(data['cores'] == new_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem)]
        sample2 = data[(data['cores'] == mid_cores) & (data['gpu'] == new_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == mid_mem)]
        sample3 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == new_cpu) & (data['mem'] == mid_mem)]
        sample4 = data[(data['cores'] == mid_cores) & (data['gpu'] == mid_gpu) & (data['cpu'] == mid_cpu) & (data['mem'] == new_mem)]

        index_list.append(sample1.index[0])
        index_list.append(sample2.index[0])
        index_list.append(sample3.index[0])
        index_list.append(sample4.index[0])

        pair1 = index_list[0:2]
        pair2 = index_list[0:1] + index_list[2:3]
        pair3 = index_list[0:1] + index_list[3:4]
        pair4 = index_list[0:1] + index_list[4:5]

        pairs = [pair1, pair2, pair3, pair4]

    return pairs

In [8]:
# pairs = return_initial_samples(data, 40)

In [9]:
def fit_lines(data, pairs, zero_list):
    flag = False
    mode = ['cores', 'gpu','cpu','mem']
    slopes = []
    for i in range(len(pairs)):
        # Fit a time and power line for each pair
        sample1 = data.iloc[pairs[i][0]]
        sample2 = data.iloc[pairs[i][1]]
        
        sample1['cores'] = sample1['cores']*100
        sample1['cpu'] = sample1['cpu']/1000
        sample1['mem'] = sample1['mem']/1000000
        sample1['gpu'] = sample1['gpu']/1000000

        sample2['cores'] = sample2['cores']*100
        sample2['cpu'] = sample2['cpu']/1000
        sample2['mem'] = sample2['mem']/1000000
        sample2['gpu'] = sample2['gpu']/1000000
        # print(sample1)
        # print(sample2)

        x1 = sample1[mode[i]]
        x2 = sample2[mode[i]]

        yt1 = sample1['observed_time']
        yt2 = sample2['observed_time']

        yp1 = sample1['observed_power']
        yp2 = sample2['observed_power']

        m_time = (yt2 - yt1)/(x2 - x1)
        c_time = yt1 - m_time*x1

        m_power = (yp2 - yp1)/(x2 - x1)
        c_power = yp1 - m_power*x1

        slope = m_time/m_power

        if abs(yp2 - yp1) < 1:
            slope = 0
            print("Power under 1W for mode {dim}, setting slope to 0".format(dim=mode[i]))
        # print("Mode : ", mode[i])
        # m_time_formated = "{:.3f}".format(m_time)
        # c_time_formated = "{:.3f}".format(c_time)
        # m_power_formated = "{:.3f}".format(m_power)
        # c_power_formated = "{:.3f}".format(c_power)
        # print(f"Time Line: y = {m_time_formated}x + {c_time_formated}")
        # print(f"Power Line: y = {m_power_formated}x + {c_power_formated}")
        # print("B/A =","{:.3f}".format( m_time/m_power))
        slopes.append(slope)
        # print("\n")
    

    slopes = [abs(x) for x in slopes]
    # change any value greater than 100 to 0 in slopes list

    # for i in range(len(slopes)):
    #     if slopes[i] > 100:
    #         slopes[i] = 0
    #         print("Slope of {dim} is too high, setting to 0".format(dim=mode[i]))
    #         #append the pairs with high slope to a list
    #         high_slope_list.append(pairs[i])
        
    # find the index of elements in zero_list
    zero_index = [mode.index(x) for x in zero_list]
    # print("Setting slope of {dim} to 0".format(dim=zero_list))
    # set the slopes of index in zero_index to 0
    for i in zero_index:
        slopes[i] = 0

    # round the slopes to 2 decimal places
    slopes_print = [round(x, 2) for x in slopes]
    print("Slopes (Cores, GPU, CPU, Memory): ", slopes_print)
    if sum(slopes) == 0:
        flag = True
    # return the index of the minimum value in the slopes list
    max_index = slopes.index(max(slopes))
    max_dim = mode[max_index]
    
    return max_index, max_dim, flag


In [10]:
# fit_lines(data, pairs, zero_list)

In [11]:
def bs(core_vals, cpu_vals, gpu_vals, mem_vals, dim, step, cores, cpu, gpu, mem):
    # Using binary search to find the optimal value for the dimension

    
    if dim == 'cores':
        vals = core_vals
        value = cores
    elif dim == 'cpu':
        vals = cpu_vals
        value = cpu
    elif dim == 'gpu':
        vals = gpu_vals
        value = gpu
    else:
        vals = mem_vals
        value = mem

    if step=='next':    
        # print("Mode: Next")
        dim_index = vals.index(value)
        # print("Dim Index: ", dim_index)
        last_index = len(vals)-1
        # print("Last Index: ", last_index)
        new_index = int(np.ceil((dim_index + last_index)/2))
        value = vals[new_index]
        vals = vals[dim_index+1:]


    else:
        # print("Mode: Prev")
        dim_index = vals.index(value)
        new_index = int(np.floor(dim_index/2))
        value = vals[new_index]
        vals = vals[:dim_index]

    return value, vals


In [12]:
def main(data, power_budget, max_count):

    index_list = []
    zero_list = []
    # high_slope_list = []
    visited_dim = []
    core_vals=[4, 8, 12] #3 possible values
    gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
    cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
    mem_vals = [665600000, 2133000000, 3199000000]

    pairs = return_initial_samples(data, power_budget)
    for i in pairs:
        index_list.append(i[0])
        index_list.append(i[1])
    print("Initial Pairs: ", pairs)
    mid_index = index_list[0]
    # print("Initial Index: ", mid_index)
    mid_power = data.iloc[mid_index]['observed_power']
    print("Initial Power: ", mid_power)
    print("\n")
    count = 1
    # while True:
    while count < max_count:
        
        max_index, max_dim, flag = fit_lines(data, pairs, zero_list)
        print("Dimension with max slope: ", max_dim)
        visited_dim.append(max_dim)
        # print("Max Dim: ", max_dim)
        if flag:
            break

        next_pair = pairs[max_index]

        if count == 1:
            curr_sample = data.iloc[next_pair[0]]
            curr_cores = curr_sample['cores']
            curr_cpu = curr_sample['cpu']
            curr_gpu = curr_sample['gpu']
            curr_mem = curr_sample['mem']
            curr_power = curr_sample['observed_power']

        # else:
        #     curr_sample = data.iloc[next_pair[1]]
        #     curr_cores = curr_sample['cores']
        #     curr_cpu = curr_sample['cpu']
        #     curr_gpu = curr_sample['gpu']
        #     curr_mem = curr_sample['mem']
        #     curr_power = curr_sample['observed_power']

        # print("Current Cores, CPU, GPU, Memory: ",curr_cores, curr_cpu, curr_gpu, curr_mem)
        # print("\n")
        # print("Current Power: ", curr_power)

        if curr_power < power_budget:
            value, vals = bs(core_vals, cpu_vals, gpu_vals, mem_vals, max_dim, 'next', curr_cores, curr_cpu, curr_gpu, curr_mem)
            print("New Dim value: ", value)
            print("Updated dim list: ", vals)
            if len(vals) == 1:
                zero_list.append(max_dim)
                print("Setting slope of {dim} to 0".format(dim=max_dim))

            if max_dim == 'cores':
                curr_cores = value
                core_vals = vals
            elif max_dim == 'cpu':
                curr_cpu = value
                cpu_vals = vals
            elif max_dim == 'gpu':
                curr_gpu = value
                gpu_vals = vals
            else:
                curr_mem = value
                mem_vals = vals

        else:
            value, vals = bs(core_vals, cpu_vals, gpu_vals, mem_vals, max_dim, 'prev', curr_cores, curr_cpu, curr_gpu, curr_mem)
            print("New Dim value: ", value)
            print("Updated dim list: ", vals)
            if len(vals) == 1:
                zero_list.append(max_dim)
                print("Setting slope of {dim} to 0".format(dim=max_dim))

            if max_dim == 'cores':
                curr_cores = value
                core_vals = vals
            elif max_dim == 'cpu':
                curr_cpu = value
                cpu_vals = vals
            elif max_dim == 'gpu':
                curr_gpu = value
                gpu_vals = vals
            else:
                curr_mem = value
                mem_vals = vals

        # retrive new index from data
        new_sample = data[(data['cores'] == int(curr_cores)) & (data['gpu'] == int(curr_gpu)) & (data['cpu'] == int(curr_cpu)) & (data['mem'] == int(curr_mem))]
        curr_power = new_sample['observed_power'].values[0]
        new_index = new_sample.index[0]
        index_list.append(new_index)
        # print("New Index at "+str(count)+" iteration: ",new_index)


        if curr_power < power_budget:
            pairs[max_index] = [next_pair[0], new_index]
        else:
            pairs[max_index] = [new_index, next_pair[1]]

        # print("Pairs at "+str(count)+" iteration: ",pairs) 
        # divide the curr_cpu by 1000, curr_mem by 1000000 and curr_gpu by 1000000
        curr_cpu_print = curr_cpu/1000
        curr_gpu_print = curr_gpu/1000000
        curr_mem_print = curr_mem/1000000
        print("Current Cores, CPU, GPU, Memory at "+str(count)+" iteration: ",curr_cores, curr_cpu_print, curr_gpu_print, curr_mem_print)
        print("Power at "+str(count)+" iteration: ",curr_power)

        print("\n")
        count+=1

    visited_dim = list(set(visited_dim))

    return index_list, len(set(index_list)), visited_dim
    

In [13]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]

In [14]:
data.sort_values(by=['observed_power'])

,cores,cpu,gpu,mem,observed_time,observed_power
20,4,422400,114750000,665600000,492.637039,14.7510
41,4,729600,114750000,665600000,477.555252,15.1520
104,4,1651200,114750000,665600000,470.851410,15.1520
167,8,422400,114750000,665600000,484.998489,15.1520
314,12,422400,114750000,665600000,483.421570,15.1520
...,...,...,...,...,...,...
400,12,1958400,1300500000,3199000000,58.941088,48.7785
127,4,2201600,1300500000,3199000000,60.644400,49.3350
253,8,1958400,1300500000,3199000000,58.787088,49.6870
421,12,2201600,1300500000,3199000000,59.179008,51.1290


In [15]:
index_list, _, _ = main(data, 33, 15)

Initial Pairs:  [[219, 366], [219, 210], [219, 282], [219, 220]]
Initial Power:  28.095


Power under 1W for mode cores, setting slope to 0
Slopes (Cores, GPU, CPU, Memory):  [0, 1.8, 1.59, 8.93]
Dimension with max slope:  mem
New Dim value:  3199000000
Updated dim list:  [3199000000]
Setting slope of mem to 0
Current Cores, CPU, GPU, Memory at 1 iteration:  8.0 1344.0 726.75 3199.0
Power at 1 iteration:  29.436


Power under 1W for mode cores, setting slope to 0
Slopes (Cores, GPU, CPU, Memory):  [0, 1.8, 1.59, 0]
Dimension with max slope:  gpu
New Dim value:  1134750000
Updated dim list:  [930750000, 1134750000, 1300500000]
Current Cores, CPU, GPU, Memory at 2 iteration:  8.0 1344.0 1134.75 3199.0
Power at 2 iteration:  41.407


Power under 1W for mode cores, setting slope to 0
Slopes (Cores, GPU, CPU, Memory):  [0, 6.0, 1.59, 0]
Dimension with max slope:  gpu
New Dim value:  930750000
Updated dim list:  [930750000]
Setting slope of gpu to 0
Current Cores, CPU, GPU, Memory at 3 itera

In [16]:
if model == 'bert':
    power_list = np.arange(10,61,1)
else:
    power_list = np.arange(10,51,1)

In [17]:
# temp_power_list = [26]

In [18]:
def pick_best_powermode(index_list, data, power_budget):
    # filter data based on the index_list
    filtered_data = data.iloc[index_list]
    # filter data based on the power budget
    filtered_data = filtered_data[filtered_data['observed_power'] <= power_budget]
    # sort the data based on the observed time
    filtered_data = filtered_data.sort_values(by='observed_time')
    # pick the first sample
    best_sample = filtered_data.iloc[0]
    return best_sample

In [19]:
test_df = data.iloc[index_list]

In [20]:
test_df

,cores,cpu,gpu,mem,observed_time,observed_power
219,8,1344000,726750000,2133000000,97.262112,28.095
366,12,1344000,726750000,2133000000,96.757072,28.100
219,8,1344000,726750000,2133000000,97.262112,28.095
210,8,1344000,1300500000,2133000000,76.530064,39.616
219,8,1344000,726750000,2133000000,97.262112,28.095
282,8,2201600,726750000,2133000000,93.919346,30.191
219,8,1344000,726750000,2133000000,97.262112,28.095
220,8,1344000,726750000,3199000000,85.293392,29.436
220,8,1344000,726750000,3199000000,85.293392,29.436
214,8,1344000,1134750000,3199000000,65.786049,41.407


In [21]:
test_df = test_df[test_df['observed_power'] <= 33]

In [22]:
test_df.sort_values(by='observed_time')

,cores,cpu,gpu,mem,observed_time,observed_power
175,8,729600,930750000,3199000000,83.337582,29.491
220,8,1344000,726750000,3199000000,85.293392,29.436
220,8,1344000,726750000,3199000000,85.293392,29.436
282,8,2201600,726750000,2133000000,93.919346,30.191
366,12,1344000,726750000,2133000000,96.757072,28.100
219,8,1344000,726750000,2133000000,97.262112,28.095
219,8,1344000,726750000,2133000000,97.262112,28.095
219,8,1344000,726750000,2133000000,97.262112,28.095
219,8,1344000,726750000,2133000000,97.262112,28.095


In [23]:
pick_best_powermode(index_list, data, 33)

cores             8.000000e+00
cpu               7.296000e+05
gpu               9.307500e+08
mem               3.199000e+09
observed_time     8.333758e+01
observed_power    2.949100e+01
Name: 175, dtype: float64

In [24]:
result_df = pd.DataFrame(columns=['cores', 'cpu', 'gpu', 'mem', 'algo_time', 'algo_power','power_budget','num_pwds','visited_dim'])
# high_slope_df = pd.DataFrame(columns=['high_slope_list','power_budget'])
avg_pwmds = []
for power_budget in power_list:
    print("Power Budget: ", power_budget)
    index_list, num_pwds, visited_dim = main(data, power_budget, 15)
    # high_slope_df.loc[len(high_slope_df)] = [high_slope_list, power_budget]

    avg_pwmds.append(num_pwds)
    print(index_list)
    try:
        best_sample = pick_best_powermode(index_list, data, power_budget)
        result_df.loc[len(result_df)] = [best_sample['cores'], best_sample['cpu'], best_sample['gpu'], best_sample['mem'], best_sample['observed_time'], best_sample['observed_power'], power_budget, num_pwds, len(visited_dim)]
    except:
        print("No Power Mode Found")
        low_pwmd = data[(data['cores'] == 4) & (data['gpu'] == 114750000) & (data['cpu'] == 422400) & (data['mem'] == 665600000)]
        if low_pwmd['observed_power'].values[0] < power_budget:
            # result_df.loc[len(result_df)] = [4, 422400, 114750000, 665600000, low_pwmd['observed_time'].values[0], low_pwmd['observed_power'].values[0], power_budget, 6, 0]
            result_df.loc[len(result_df)] = [0, 0, 0, 0, 0, 0, power_budget, 0, 0]

        else:
            result_df.loc[len(result_df)] = [0, 0, 0, 0, 0, 0, power_budget, 0, 0]
        # high_slope_df.loc[len(high_slope_df)] = [0, power_budget]

print("Average Power Modes: ", np.mean(avg_pwmds))

Power Budget:  10
Initial Pairs:  [[219, 72], [219, 228], [219, 156], [219, 221]]
Initial Power:  28.095


Slopes (Cores, GPU, CPU, Memory):  [2.16, 30.99, 6.98, 13.82]
Dimension with max slope:  gpu
New Dim value:  318750000
Updated dim list:  [114750000, 318750000, 522750000]
Current Cores, CPU, GPU, Memory at 1 iteration:  8.0 1344.0 318.75 2133.0
Power at 1 iteration:  21.08


Slopes (Cores, GPU, CPU, Memory):  [2.16, 58.97, 6.98, 13.82]
Dimension with max slope:  gpu
New Dim value:  114750000
Updated dim list:  [114750000]
Setting slope of gpu to 0
Current Cores, CPU, GPU, Memory at 2 iteration:  8.0 1344.0 114.75 2133.0
Power at 2 iteration:  16.159


Power under 1W for mode gpu, setting slope to 0
Slopes (Cores, GPU, CPU, Memory):  [2.16, 0, 6.98, 13.82]
Dimension with max slope:  mem
New Dim value:  665600000
Updated dim list:  [665600000]
Setting slope of mem to 0
Current Cores, CPU, GPU, Memory at 3 iteration:  8.0 1344.0 114.75 665.6
Power at 3 iteration:  15.254


Power und

In [25]:
def find_optimal_pm(df, pmax):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    df = df[df['observed_power'] <= pmax]
    df = df.sort_values(by='observed_time', ascending=True)
    #return top1 powermode, time and power
    return df.iloc[0]['cores'], df.iloc[0]['cpu'], df.iloc[0]['gpu'], df.iloc[0]['mem'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power']

In [26]:
optimal_df = pd.DataFrame(columns=['power_budget', 'cores', 'cpu', 'gpu', 'mem', 'optim_time', 'optim_power'])
for power_budget in power_list:
    try:
        cores, cpu, gpu, mem, algo_time, algo_power = find_optimal_pm(data, power_budget)
        optimal_df.loc[len(optimal_df)] = [power_budget, cores, cpu, gpu, mem, algo_time, algo_power]
    except:
        print("No Power Mode Found")
        optimal_df.loc[len(optimal_df)] = [power_budget, 0, 0, 0, 0, 0, 0]

No Power Mode Found
No Power Mode Found
No Power Mode Found
No Power Mode Found
No Power Mode Found


In [27]:
# merge on power_budget
final_df = pd.merge(result_df, optimal_df, on='power_budget', suffixes=('_algo', '_optimal'))

In [28]:
data[(data['cores'] == 4) & (data['gpu'] == 114750000) & (data['cpu'] == 422400) & (data['mem'] == 665600000)]

,cores,cpu,gpu,mem,observed_time,observed_power
20,4,422400,114750000,665600000,492.637039,14.751


In [29]:
data.sort_values(by=['observed_power'])

,cores,cpu,gpu,mem,observed_time,observed_power
20,4,422400,114750000,665600000,492.637039,14.7510
41,4,729600,114750000,665600000,477.555252,15.1520
104,4,1651200,114750000,665600000,470.851410,15.1520
167,8,422400,114750000,665600000,484.998489,15.1520
314,12,422400,114750000,665600000,483.421570,15.1520
...,...,...,...,...,...,...
400,12,1958400,1300500000,3199000000,58.941088,48.7785
127,4,2201600,1300500000,3199000000,60.644400,49.3350
253,8,1958400,1300500000,3199000000,58.787088,49.6870
421,12,2201600,1300500000,3199000000,59.179008,51.1290


In [30]:
final_df

,cores_algo,cpu_algo,gpu_algo,mem_algo,algo_time,algo_power,power_budget,num_pwds,visited_dim,cores_optimal,cpu_optimal,gpu_optimal,mem_optimal,optim_time,optim_power
0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000,10.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000
1,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000,11.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000
2,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000,12.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000
3,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000,13.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000
4,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000,14.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.0000
5,4.0,422400.0,1.147500e+08,6.656000e+08,492.637039,14.7510,15.0,10.0,4.0,4.0,422400.0,1.147500e+08,6.656000e+08,492.637039,14.7510
6,8.0,1958400.0,1.147500e+08,6.656000e+08,470.392365,15.7540,16.0,10.0,4.0,12.0,1958400.0,1.147500e+08,6.656000e+08,469.961273,15.9540
7,8.0,1344000.0,1.147500e+08,3.199000e+09,465.590149,16.9720,17.0,10.0,4.0,4.0,422400.0,3.187500e+08,6.656000e+08,316.508636,16.3560
8,8.0,1958400.0,1.147500e+08,3.199000e+09,464.100006,17.3730,18.0,10.0,4.0,4.0,1344000.0,3.187500e+08,6.656000e+08,240.882660,17.7725
9,12.0,2201600.0,1.147500e+08,3.199000e+09,464.036285,18.5690,19.0,10.0,4.0,4.0,1344000.0,5.227500e+08,6.656000e+08,214.160271,18.9750


In [31]:
final_df.to_csv(f'{model}_bs_lr_exp_dim.csv', index=False)